In [ ]:
import sys
sys.path.append('py')
from importlib import reload # Python 3.4+
import parse_fit
reload(parse_fit)

fname = "data/paddle_hcc.fit"
parse_fit.list_fields(fname)

In [ ]:
import sys
sys.path.append('py')
from importlib import reload # Python 3.4+
import parse_fit
reload(parse_fit)

fname = "data/paddle_hcc.fit"
laps_df, points_df = parse_fit.get_dataframes(fname)
# print('LAPS:')
# print(laps_df.head())
print('\nPOINTS:')
print(points_df.head())


mlat = points_df['lat'].mean()
print(mlat)


In [ ]:

import branca.colormap as cm

speed = cm.LinearColormap( ["red", "yellow", "green"], vmin=1.5, vmax=4.1, caption="speed")

speed

import folium

# creat map of latitude and longitude in points_df
mlat = points_df['lat'].mean()
mlon = points_df['lon'].mean()
print (mlat, mlon)
m = folium.Map(location=[mlat, mlon], zoom_start=16, tiles="cartodb positron")

trail = points_df[['lat', 'lon']].values.tolist()
# folium.PolyLine(trail, color="red", weight=2.5, opacity=1).add_to(m)
folium.ColorLine(trail, points_df['enhanced_speed'], weight=4, opacity=1, colormap=speed).add_to(m)
m

from rdp import rdp
import numpy as np

# extract lon and lat from points_df and convert to numpy array
lon = points_df['lon'].values
lat = points_df['lat'].values
# print (lon.shape, lat.shape)

# combine lat and lon into array

arr = points_df[['lat', 'lon']].to_numpy()
arr.shape
tpmask = rdp(arr, epsilon = 0.001, return_mask=True)
# print (tpmask)
timestamps = points_df['timestamp'].values
timestamps[tpmask]


In [ ]:


# creat map of latitude and longitude in points_df
mlat = points_df['lat'].mean()
mlon = points_df['lon'].mean()
print (mlat, mlon)
# m = folium.Map(location=[points_df['position_lat'].mean(), points_df['position_long'].mean()], zoom_start=14)
# m

In [ ]:
arr = np.array([1, 1, 2, 2, 3, 3, 4, 4]).reshape(4, 2)
arr.shape

In [ ]:
# return tail(x, -2) - head(x, -2)


def curvature(lng, lat):

  def first_diff(x) :
    return [x[i] - x[i-2] for i in range(2, len(x))]
      
  def second_diff(x) :
    return [x[i] - 2 * x[i-1] + x[i-2] for i in range(2, len(x))]

  def range_len(x) :
    return range(len(x))
  
  x  = lng
  y  = lat
  x1 = first_diff(x)
  y1 = first_diff(y)
  x2 = second_diff(x)
  y2 = second_diff(y)
  num = [abs(x1[i] * y2[i] - y1[i] * x2[i]) for i in range_len(x1)]
  denom = [((x1[i]**2 + y1[i]**2)**3)**0.5 for i in range_len(x1)]
  # z  =  [abs(x1[i] * y2[i] - y1[i] * x2[i]) / ((x1[i]^2 + y1[i]^2)^3)**0.5 for i in range(len(x1))]
  # return [0, z, 0]
  return [0, [num[i] / denom[i] for i in range(len(num))], 0]

lng = [1, 2, 3, 2, 5]
lat = [1,3,1,4,2]
print(lng[2:])

print(first_diff(lng))
print(second_diff(lng))
curvature(lng, lat)

In [ ]:
points_df

In [ ]:
points_df
# summarise points_df
points_df.describe()

# compute round of lat and lon of points_df
points_df['round_lat'] = points_df['lat'].round(3)
points_df['round_lon'] = points_df['lon'].round(3)
points_df
# compute rolling change in round_lat and round_lon
points_df['delta_lat'] = points_df['round_lat'].diff()
points_df['delta_lon'] = points_df['round_lon'].diff()



spatial_hash(points_df)



In [ ]:
# compute quantiles of enhanced_speed
points_df['enhanced_speed'].quantile([0.1, 0.5, 0.9])

import matplotlib
# plot enhanced_speed and add plot title
points_df['enhanced_speed'].plot().set_title('enhanced_speed')
# matplotlib.pyplot.title('enhanced_speed')

In [ ]:

import matplotlib
import datetime
import turnpoints
import pandas as pd
# p = points_df['enhanced_speed'].plot().set_title('Speed (m/s)')
points_df = turnpoints.read_garmin(fname)
_, _, tps_ = turnpoints.create_map(points_df)
tps_
    
    # for tp in tps_:
    #     matplotlib.pyplot.axvline(x=tp, color='r', linestyle='--')

    # # compute average speed between turnpoints
avs = []
# points_df['timestamp'] = points_df['timestamp'].to_timestamp()
# tps_ = [pd.to_datetime(tp) for tp in tps_]
tps_ = pd.DataFrame(data = {"timestamp": tps_})
tps_['timestamp'] = pd.to_datetime(tps_['timestamp'], utc=True)
points_df['timestamp'] = pd.to_datetime(points_df['timestamp'], utc=True)
# print number of rows of tps_

# print(tps_['timestamp'].dtypes)
# points_df['timestamp'].dtypes = "datetime64"
# print(points_df['timestamp'].dtypes)

for i in range(tps_.shape[0] - 1):
    ss = points_df[
        points_df['timestamp'] >= tps_['timestamp'][i] 
        # points_df['timestamp'] < tps_[i+1]
    ]
    mean_speed = ss['enhanced_speed'].mean() 
    avs.append(mean_speed)
print(avs)